# Sales Data Preprocessing Notebook

This notebook contains comprehensive data preprocessing steps for the sales data dashboard.

## Overview
- Load and explore sales data
- Handle missing values and duplicates
- Normalize categories and extract time features
- Save cleaned data for dashboard consumption


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("Libraries imported successfully!")


## 1. Load and Explore Data


In [ ]:
# Load the sales data
df = pd.read_csv('../data/raw/sales_data.csv')

print(f"Dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print("\nFirst 5 rows:")
df.head()


In [ ]:
# Basic data information
print("Dataset Info:")
print(df.info())
print("\nData Types:")
print(df.dtypes)
print("\nMissing Values:")
print(df.isnull().sum())
print("\nDuplicate Rows:")
print(f"Total duplicates: {df.duplicated().sum()}")


## 2. Data Cleaning and Preprocessing


In [ ]:
# Create a copy for cleaning
df_clean = df.copy()

# Remove duplicates
df_clean = df_clean.drop_duplicates()
print(f"After removing duplicates: {df_clean.shape}")

# Handle missing values
print("\nMissing values before cleaning:")
print(df_clean.isnull().sum())

# Fill missing values
df_clean['CUSTOMERNAME'] = df_clean['CUSTOMERNAME'].fillna('Unknown Customer')
df_clean['COUNTRY'] = df_clean['COUNTRY'].fillna('Unknown Country')
df_clean['PRODUCTLINE'] = df_clean['PRODUCTLINE'].fillna('Unknown Product')
df_clean['STATUS'] = df_clean['STATUS'].fillna('Unknown Status')

# For numeric columns, fill with median
df_clean['SALES'] = df_clean['SALES'].fillna(df_clean['SALES'].median())
df_clean['QUANTITYORDERED'] = df_clean['QUANTITYORDERED'].fillna(df_clean['QUANTITYORDERED'].median())

print("\nMissing values after cleaning:")
print(df_clean.isnull().sum())


In [ ]:
# Convert ORDERDATE to datetime
df_clean['ORDERDATE'] = pd.to_datetime(df_clean['ORDERDATE'], errors='coerce')

# Extract time features
df_clean['YEAR'] = df_clean['ORDERDATE'].dt.year
df_clean['MONTH'] = df_clean['ORDERDATE'].dt.month
df_clean['QUARTER'] = df_clean['ORDERDATE'].dt.quarter
df_clean['DAY_OF_WEEK'] = df_clean['ORDERDATE'].dt.day_name()
df_clean['MONTH_NAME'] = df_clean['ORDERDATE'].dt.month_name()

print("Time features extracted successfully!")
print(f"Date range: {df_clean['ORDERDATE'].min()} to {df_clean['ORDERDATE'].max()}")


In [ ]:
# Normalize categorical data
df_clean['PRODUCTLINE'] = df_clean['PRODUCTLINE'].str.strip().str.title()
df_clean['COUNTRY'] = df_clean['COUNTRY'].str.strip().str.title()
df_clean['CUSTOMERNAME'] = df_clean['CUSTOMERNAME'].str.strip().str.title()
df_clean['STATUS'] = df_clean['STATUS'].str.strip().str.title()

# Ensure numeric columns are properly typed
df_clean['SALES'] = pd.to_numeric(df_clean['SALES'], errors='coerce')
df_clean['QUANTITYORDERED'] = pd.to_numeric(df_clean['QUANTITYORDERED'], errors='coerce')
df_clean['ORDERNUMBER'] = pd.to_numeric(df_clean['ORDERNUMBER'], errors='coerce')

print("Data normalization completed!")


## 3. Data Analysis and Visualization


In [ ]:
# Create comprehensive visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Sales Data Analysis Dashboard', fontsize=16, fontweight='bold')

# 1. Sales by Product Line
product_sales = df_clean.groupby('PRODUCTLINE')['SALES'].sum().sort_values(ascending=False)
colors1 = plt.cm.Set3(np.linspace(0, 1, len(product_sales)))
bars1 = axes[0,0].bar(product_sales.index, product_sales.values, color=colors1)
axes[0,0].set_title('Total Sales by Product Line', fontweight='bold', pad=20)
axes[0,0].set_xlabel('Product Line')
axes[0,0].set_ylabel('Total Sales ($)')
axes[0,0].tick_params(axis='x', rotation=45)
# Add value labels on bars
for bar in bars1:
    height = bar.get_height()
    axes[0,0].text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                   f'${height:,.0f}', ha='center', va='bottom', fontsize=9)

# 2. Sales by Country (Top 10)
country_sales = df_clean.groupby('COUNTRY')['SALES'].sum().sort_values(ascending=False).head(10)
colors2 = plt.cm.viridis(np.linspace(0, 1, len(country_sales)))
bars2 = axes[0,1].bar(country_sales.index, country_sales.values, color=colors2)
axes[0,1].set_title('Top 10 Countries by Sales', fontweight='bold', pad=20)
axes[0,1].set_xlabel('Country')
axes[0,1].set_ylabel('Total Sales ($)')
axes[0,1].tick_params(axis='x', rotation=45)
# Add value labels on bars
for bar in bars2:
    height = bar.get_height()
    axes[0,1].text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                   f'${height:,.0f}', ha='center', va='bottom', fontsize=9)

# 3. Sales Trend Over Time
monthly_sales = df_clean.groupby(['YEAR', 'MONTH'])['SALES'].sum().reset_index()
monthly_sales['DATE'] = pd.to_datetime(monthly_sales[['YEAR', 'MONTH']].assign(DAY=1))
axes[1,0].plot(monthly_sales['DATE'], monthly_sales['SALES'], marker='o', linewidth=2, markersize=6, color='#2E86AB')
axes[1,0].set_title('Sales Trend Over Time', fontweight='bold', pad=20)
axes[1,0].set_xlabel('Date')
axes[1,0].set_ylabel('Total Sales ($)')
axes[1,0].tick_params(axis='x', rotation=45)
axes[1,0].grid(True, alpha=0.3)
# Add trend line
z = np.polyfit(range(len(monthly_sales)), monthly_sales['SALES'], 1)
p = np.poly1d(z)
axes[1,0].plot(monthly_sales['DATE'], p(range(len(monthly_sales))), "r--", alpha=0.8, linewidth=2)

# 4. Order Status Distribution
status_counts = df_clean['STATUS'].value_counts()
colors4 = plt.cm.Pastel1(np.linspace(0, 1, len(status_counts)))
wedges, texts, autotexts = axes[1,1].pie(status_counts.values, labels=status_counts.index, 
                                        autopct='%1.1f%%', colors=colors4, startangle=90)
axes[1,1].set_title('Order Status Distribution', fontweight='bold', pad=20)
# Enhance text appearance
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')

plt.tight_layout()
plt.show()

# Print summary statistics
print("="*60)
print("VISUALIZATION SUMMARY")
print("="*60)
print(f"📊 Total Product Lines: {len(product_sales)}")
print(f"🌍 Countries Analyzed: {len(country_sales)}")
print(f"📅 Time Period: {monthly_sales['DATE'].min().strftime('%B %Y')} to {monthly_sales['DATE'].max().strftime('%B %Y')}")
print(f"📋 Order Statuses: {len(status_counts)}")
print("="*60)


In [ ]:
# Advanced Analytics and Insights
print("="*70)
print("ADVANCED SALES ANALYTICS")
print("="*70)

# 1. Top Performing Analysis
print("\n🏆 TOP PERFORMERS:")
print("-" * 40)

# Top Product Line
top_product = df_clean.groupby('PRODUCTLINE')['SALES'].sum().idxmax()
top_product_sales = df_clean.groupby('PRODUCTLINE')['SALES'].sum().max()
print(f"🥇 Best Product Line: {top_product} (${top_product_sales:,.2f})")

# Top Country
top_country = df_clean.groupby('COUNTRY')['SALES'].sum().idxmax()
top_country_sales = df_clean.groupby('COUNTRY')['SALES'].sum().max()
print(f"🌍 Top Country: {top_country} (${top_country_sales:,.2f})")

# Top Customer
top_customer = df_clean.groupby('CUSTOMERNAME')['SALES'].sum().idxmax()
top_customer_sales = df_clean.groupby('CUSTOMERNAME')['SALES'].sum().max()
print(f"👤 Top Customer: {top_customer} (${top_customer_sales:,.2f})")

# 2. Performance Metrics
print("\n📈 PERFORMANCE METRICS:")
print("-" * 40)

# Average Order Value
avg_order_value = df_clean['SALES'].mean()
print(f"💰 Average Order Value: ${avg_order_value:,.2f}")

# Total Revenue
total_revenue = df_clean['SALES'].sum()
print(f"💵 Total Revenue: ${total_revenue:,.2f}")

# Total Orders
total_orders = df_clean['ORDERNUMBER'].nunique()
print(f"📦 Total Orders: {total_orders:,}")

# Total Quantity
total_quantity = df_clean['QUANTITYORDERED'].sum()
print(f"📊 Total Quantity Sold: {total_quantity:,}")

# 3. Time-based Analysis
print("\n⏰ TIME-BASED ANALYSIS:")
print("-" * 40)

# Best Month
monthly_performance = df_clean.groupby(['YEAR', 'MONTH'])['SALES'].sum()
best_month = monthly_performance.idxmax()
best_month_sales = monthly_performance.max()
print(f"📅 Best Month: {best_month[1]}/{best_month[0]} (${best_month_sales:,.2f})")

# Worst Month
worst_month = monthly_performance.idxmin()
worst_month_sales = monthly_performance.min()
print(f"📅 Worst Month: {worst_month[1]}/{worst_month[0]} (${worst_month_sales:,.2f})")

# Year-over-Year Growth
yearly_sales = df_clean.groupby('YEAR')['SALES'].sum()
if len(yearly_sales) > 1:
    current_year = yearly_sales.index[-1]
    previous_year = yearly_sales.index[-2]
    current_sales = yearly_sales[current_year]
    previous_sales = yearly_sales[previous_year]
    yoy_growth = ((current_sales - previous_sales) / previous_sales) * 100
    print(f"📈 YoY Growth ({previous_year} to {current_year}): {yoy_growth:+.1f}%")

# 4. Customer Analysis
print("\n👥 CUSTOMER ANALYSIS:")
print("-" * 40)

# Customer count
unique_customers = df_clean['CUSTOMERNAME'].nunique()
print(f"👤 Unique Customers: {unique_customers:,}")

# Average orders per customer
avg_orders_per_customer = total_orders / unique_customers
print(f"📦 Average Orders per Customer: {avg_orders_per_customer:.1f}")

# Customer concentration (top 10 customers)
top_10_customers = df_clean.groupby('CUSTOMERNAME')['SALES'].sum().nlargest(10)
top_10_percentage = (top_10_customers.sum() / total_revenue) * 100
print(f"🎯 Top 10 Customers Revenue Share: {top_10_percentage:.1f}%")

# 5. Product Analysis
print("\n🏷️ PRODUCT ANALYSIS:")
print("-" * 40)

# Product diversity
unique_products = df_clean['PRODUCTLINE'].nunique()
print(f"🏷️ Product Lines: {unique_products}")

# Product concentration
top_product_share = (top_product_sales / total_revenue) * 100
print(f"🥇 Top Product Share: {top_product_share:.1f}%")

# Average sales per product
avg_sales_per_product = total_revenue / unique_products
print(f"📊 Average Sales per Product Line: ${avg_sales_per_product:,.2f}")

print("\n" + "="*70)


In [ ]:
# Data Quality Assessment and Validation
print("="*70)
print("DATA QUALITY ASSESSMENT")
print("="*70)

# 1. Data Completeness Check
print("\n✅ DATA COMPLETENESS:")
print("-" * 40)

total_records = len(df_clean)
print(f"📊 Total Records: {total_records:,}")

# Check for missing values after cleaning
missing_values = df_clean.isnull().sum()
missing_percentage = (missing_values / total_records) * 100

print("\nMissing Values Analysis:")
for column, missing_count in missing_values.items():
    if missing_count > 0:
        print(f"⚠️  {column}: {missing_count} missing ({missing_percentage[column]:.1f}%)")
    else:
        print(f"✅ {column}: No missing values")

# 2. Data Consistency Check
print("\n🔍 DATA CONSISTENCY:")
print("-" * 40)

# Check for negative sales
negative_sales = (df_clean['SALES'] < 0).sum()
print(f"💰 Negative Sales: {negative_sales} records")

# Check for zero sales
zero_sales = (df_clean['SALES'] == 0).sum()
print(f"💰 Zero Sales: {zero_sales} records")

# Check for negative quantities
negative_quantities = (df_clean['QUANTITYORDERED'] < 0).sum()
print(f"📦 Negative Quantities: {negative_quantities} records")

# Check for zero quantities
zero_quantities = (df_clean['QUANTITYORDERED'] == 0).sum()
print(f"📦 Zero Quantities: {zero_quantities} records")

# 3. Data Range Analysis
print("\n📈 DATA RANGE ANALYSIS:")
print("-" * 40)

print(f"💰 Sales Range: ${df_clean['SALES'].min():,.2f} - ${df_clean['SALES'].max():,.2f}")
print(f"📦 Quantity Range: {df_clean['QUANTITYORDERED'].min():,} - {df_clean['QUANTITYORDERED'].max():,}")
print(f"📅 Date Range: {df_clean['ORDERDATE'].min().strftime('%Y-%m-%d')} to {df_clean['ORDERDATE'].max().strftime('%Y-%m-%d')}")

# 4. Outlier Detection
print("\n🎯 OUTLIER DETECTION:")
print("-" * 40)

# Sales outliers using IQR method
Q1_sales = df_clean['SALES'].quantile(0.25)
Q3_sales = df_clean['SALES'].quantile(0.75)
IQR_sales = Q3_sales - Q1_sales
lower_bound_sales = Q1_sales - 1.5 * IQR_sales
upper_bound_sales = Q3_sales + 1.5 * IQR_sales

sales_outliers = df_clean[(df_clean['SALES'] < lower_bound_sales) | (df_clean['SALES'] > upper_bound_sales)]
print(f"💰 Sales Outliers: {len(sales_outliers)} records ({len(sales_outliers)/total_records*100:.1f}%)")

# Quantity outliers
Q1_qty = df_clean['QUANTITYORDERED'].quantile(0.25)
Q3_qty = df_clean['QUANTITYORDERED'].quantile(0.75)
IQR_qty = Q3_qty - Q1_qty
lower_bound_qty = Q1_qty - 1.5 * IQR_qty
upper_bound_qty = Q3_qty + 1.5 * IQR_qty

qty_outliers = df_clean[(df_clean['QUANTITYORDERED'] < lower_bound_qty) | (df_clean['QUANTITYORDERED'] > upper_bound_qty)]
print(f"📦 Quantity Outliers: {len(qty_outliers)} records ({len(qty_outliers)/total_records*100:.1f}%)")

# 5. Data Distribution Summary
print("\n📊 DATA DISTRIBUTION SUMMARY:")
print("-" * 40)

print("Sales Distribution:")
print(f"  Mean: ${df_clean['SALES'].mean():,.2f}")
print(f"  Median: ${df_clean['SALES'].median():,.2f}")
print(f"  Std Dev: ${df_clean['SALES'].std():,.2f}")

print("\nQuantity Distribution:")
print(f"  Mean: {df_clean['QUANTITYORDERED'].mean():.1f}")
print(f"  Median: {df_clean['QUANTITYORDERED'].median():.1f}")
print(f"  Std Dev: {df_clean['QUANTITYORDERED'].std():.1f}")

# 6. Final Data Quality Score
print("\n🏆 DATA QUALITY SCORE:")
print("-" * 40)

quality_score = 100
if missing_values.sum() > 0:
    quality_score -= (missing_values.sum() / (total_records * len(df_clean.columns))) * 50
if negative_sales > 0 or negative_quantities > 0:
    quality_score -= 10
if len(sales_outliers) > total_records * 0.05:  # More than 5% outliers
    quality_score -= 5

print(f"📊 Overall Data Quality Score: {quality_score:.1f}/100")
if quality_score >= 90:
    print("✅ Excellent data quality!")
elif quality_score >= 80:
    print("✅ Good data quality!")
elif quality_score >= 70:
    print("⚠️  Fair data quality - some issues detected")
else:
    print("❌ Poor data quality - significant issues detected")

print("\n" + "="*70)


In [ ]:
# Save cleaned data and generate final summary
print("="*70)
print("FINAL DATA EXPORT AND SUMMARY")
print("="*70)

# Save the cleaned data
output_path = '../data/processed/sales_cleaned.csv'
df_clean.to_csv(output_path, index=False)

print(f"\n💾 CLEANED DATA SAVED:")
print(f"📁 Location: {output_path}")
print(f"📊 Records: {len(df_clean):,}")
print(f"📋 Columns: {len(df_clean.columns)}")

# Display final data sample
print(f"\n📋 FINAL DATA SAMPLE:")
print("-" * 40)
print(df_clean.head())

# Final summary statistics
print(f"\n📈 FINAL SUMMARY STATISTICS:")
print("-" * 40)
print(f"📊 Dataset Shape: {df_clean.shape}")
print(f"💰 Total Sales: ${df_clean['SALES'].sum():,.2f}")
print(f"📦 Total Orders: {df_clean['ORDERNUMBER'].nunique():,}")
print(f"👤 Unique Customers: {df_clean['CUSTOMERNAME'].nunique():,}")
print(f"🌍 Countries: {df_clean['COUNTRY'].nunique()}")
print(f"🏷️ Product Lines: {df_clean['PRODUCTLINE'].nunique()}")
print(f"📅 Date Range: {df_clean['ORDERDATE'].min().strftime('%Y-%m-%d')} to {df_clean['ORDERDATE'].max().strftime('%Y-%m-%d')}")

# Column information
print(f"\n📋 FINAL COLUMN INFORMATION:")
print("-" * 40)
for i, col in enumerate(df_clean.columns, 1):
    dtype = df_clean[col].dtype
    non_null = df_clean[col].count()
    print(f"{i:2d}. {col:<20} | Type: {str(dtype):<12} | Non-null: {non_null:,}")

print(f"\n✅ DATA PREPROCESSING COMPLETED SUCCESSFULLY!")
print("="*70)
print("🎯 The cleaned data is now ready for:")
print("   • Backend API consumption")
print("   • Frontend dashboard visualization")
print("   • AI/ML model training")
print("   • Business intelligence analysis")
print("="*70)
